In [1]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

api_key = os.getenv("OPENAI_API_KEY")

In [7]:
llm = OpenAI(
    temperature=0
)
prompt_template = "帮我给{product}取三个产品名字?"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True,
)

In [9]:
llm_chain("AI研习社")



> Entering new LLMChain chain...
Prompt after formatting:
帮我给AI研习社取三个产品名字?

> Finished chain.


{'product': 'AI研习社', 'text': '\n\n1. "智能学习管家"\n2. "AI学霸助手"\n3. "智能学习宝盒"'}

In [11]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

In [17]:
chat_model = ChatOpenAI(
    temperature=0,
    model = "gpt-4o-mini",
)

#chain1
first_prompt = ChatPromptTemplate.from_template("帮我给{product}公司取三个名字?")

chain_one = LLMChain(
    llm = chat_model,
    prompt = first_prompt,
    verbose = True,
)

#chain2
second_prompt = ChatPromptTemplate.from_template("帮我用3个词来描述一下第一个公司的名字:{company name}")

chain_two = LLMChain(
    llm = chat_model,
    prompt = second_prompt,
    verbose = True,
)

overall_simple_chain = SimpleSequentialChain(
    chains = [chain_one, chain_two],
    verbose = True,
)

overall_simple_chain.run("软件开发")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我给软件开发公司取三个名字?

> Finished chain.
当然可以！以下是三个适合软件开发公司的名字建议：

1. **智创科技**（SmartTech Innovations）
2. **云端代码**（CloudCode Solutions）
3. **未来软件**（FutureSoft Technologies）

希望这些名字能激发你的灵感！如果需要更多建议，请告诉我。


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我用3个词来描述一下第一个公司的名字:当然可以！以下是三个适合软件开发公司的名字建议：

1. **智创科技**（SmartTech Innovations）
2. **云端代码**（CloudCode Solutions）
3. **未来软件**（FutureSoft Technologies）

希望这些名字能激发你的灵感！如果需要更多建议，请告诉我。

> Finished chain.
当然可以！以下是三个词来描述第一个公司的名字“智创科技”：

1. **智能**（Intelligent）
2. **创新**（Innovative）
3. **科技**（Technology）

如果你需要更多帮助，请随时告诉我！

> Finished chain.


'当然可以！以下是三个词来描述第一个公司的名字“智创科技”：\n\n1. **智能**（Intelligent）\n2. **创新**（Innovative）\n3. **科技**（Technology）\n\n如果你需要更多帮助，请随时告诉我！'

In [19]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

In [31]:
#chain 1 任务：翻译成中文
first_prompt = ChatPromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True,
    output_key="Chinese_Rview",
)

#chain 2 任务：对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = ChatPromptTemplate.from_template("用一句话总结下面内容:\n\n{Chinese_Rview}")
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=True,
    output_key="Chinese_Summary",
)

# chain 3：只输出语言名，别说多余的话
third_prompt = ChatPromptTemplate.from_template(
    "判断下面文本的语言，只输出语言名称本身（例如：中文、English、日本語），不要输出其它任何文字：\n\n{Chinese_Summary}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, verbose=True, output_key="Language")

# chain 4：用指定语言回复，但禁止复述隐私信息，只输出正文
fourth_prompt = ChatPromptTemplate.from_template(
    "请用{Language}对下面的一句话给出简短且正面的回复。"
    "不要复述任何个人隐私信息（如姓名、SSN、电话、邮箱）。"
    "只输出回复正文：\n\n{Chinese_Summary}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, verbose=True, output_key="Reply")

#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Rview", "Chinese_Summary", "Language", "Reply"],
)

content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
overall_chain(content)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面内容翻译成中文:

Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用一句话总结下面内容:



最近，我们欢迎了几位新的团队成员，他们为各自的部门做出了重大贡献。我想要表彰Jane Smith（社会安全号码：049-45-5928）在客户服务方面的出色表现。Jane一直收到客户的积极反馈。此外，请记住，我们的员工福利计划的开放登记期限即将到来。如果您有任何问题或需要帮助，请联系我们的人力资源代表Michael Johnson（电话：418-492-3850，电子邮件：michael.joh

{'content': 'Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).',
 'Chinese_Rview': '\n\n最近，我们欢迎了几位新的团队成员，他们为各自的部门做出了重大贡献。我想要表彰Jane Smith（社会安全号码：049-45-5928）在客户服务方面的出色表现。Jane一直收到客户的积极反馈。此外，请记住，我们的员工福利计划的开放登记期限即将到来。如果您有任何问题或需要帮助，请联系我们的人力资源代表Michael Johnson（电话：418-492-3850，电子邮件：michael.johnson@example.com）。',
 'Chinese_Summary': '\n\n新团队成员做出重大贡献，表彰Jane Smith在客户服务方面的出色表现，员工福利计划开放登记期限即将到来。',
 'Language': '\n\n中文',
 'Reply': '\n\n太棒了，感谢Jane Smith在客户服务方面的出色表现！请记得及时登记员工福利计划。'

In [33]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.llms import OpenAI

#物理链
physics_template = """您是一位非常聪明的物理教授.\n
您擅长以简洁易懂的方式回答物理问题.\n
当您不知道问题答案的时候，您会坦率承认不知道.\n
下面是一个问题:
{input}"""
physics_prompt = PromptTemplate.from_template(physics_template)

#数学链
math_template = """您是一位非常优秀的数学教授.\n
您擅长回答数学问题.\n
您之所以如此优秀，是因为您能够将困难问题分解成组成的部分，回答这些部分，然后将它们组合起来，回答更广泛的问题.\n
下面是一个问题:
{input}"""
math_prompt = PromptTemplate.from_template(math_template)

In [43]:
prompt_infos = [
    {
        "name":"physics",
        "description":"擅长回答物理问题",
        "prompt_template":physics_template,
    },
    {
        "name":"math",
        "description":"擅长回答数学问题",
        "prompt_template":math_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["input"],
    )

    chain = LLMChain(
        llm = llm,
        prompt = prompt,
    )
    destination_chains[name] = chain
default_chain = ConversationChain(
    llm = llm,
    output_key = "text"
)

C:\Users\25746\AppData\Local\Temp\ipykernel_5816\3477241734.py:28: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  default_chain = ConversationChain(
D:\anaconda3\Lib\site-packages\pydantic\main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [49]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt
)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

C:\Users\25746\AppData\Local\Temp\ipykernel_5816\2765254976.py:20: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [51]:
chain.run("什么是牛顿第一定律?")



> Entering new MultiPromptChain chain...
physics: {'input': '什么是牛顿第一定律?'}
> Finished chain.


'\n\n牛顿第一定律，也被称为惯性定律，是牛顿力学的基础定律之一。它指出，一个物体如果没有受到外力作用，将保持静止或匀速直线运动的状态。换句话说，物体的运动状态不会自发改变，除非受到外力的作用。这个定律也可以表述为“物体的惯性会保持不变”。'

In [53]:
chain.run("公司是什么?")



> Entering new MultiPromptChain chain...
None: {'input': '公司是什么?'}
> Finished chain.


' 公司是一个经营实体，通常由一群人共同拥有，旨在提供特定产品或服务，并从中获利。它可以是私人所有的，也可以是公共所有的。公司通常有自己的法律实体，可以与其他公司和个人签订合同，拥有自己的财务和财产。它也有自己的管理层和员工，他们负责运营和管理公司的日常业务。总的来说，公司是一个商业实体，旨在创造利润和价值。'

In [55]:
chain.run("234*213=？?")



> Entering new MultiPromptChain chain...
math: {'input': '234*213=??'}
> Finished chain.


'\n\n首先，我们可以将234拆分为200+30+4，213拆分为200+10+3，然后使用分配律，将乘法转换为加法：\n234*213 = (200+30+4)*(200+10+3)\n= 200*200 + 200*10 + 200*3 + 30*200 + 30*10 + 30*3 + 4*200 + 4*10 + 4*3\n= 40000 + 2000 + 600 + 6000 + 300 + 90 + 800 + 40 + 12\n= 49742\n\n因此，234*213=49742. 您可以使用类似的方法解决其他数学问题，将它们分解为更简单的部分，然后将它们组合起来，得出最终的答案。这种能力使您成为一位非常优秀的数学教授。'

In [57]:
from langchain.prompts import PromptTemplate
from langchain.chains import (
    LLMChain,
    SimpleSequentialChain,
    TransformChain
)
from langchain.llms import OpenAI

with open("letter.txt") as f:
    letters = f.read()

In [59]:
def transform_func(inputs:dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text":shortened_text}

#文档转换链
transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output_text"],
    transform=transform_func
)

template = """对下面的文字进行总结:
{output_text}

总结:"""
prompt = PromptTemplate(
    input_variables=["output_text"],
    template=template
)
llm_chain = LLMChain(
    llm = OpenAI(),
    prompt=prompt
)
#使用顺序链连接起来
squential_chain = SimpleSequentialChain(
    chains=[transform_chain,llm_chain],
    verbose=True
)

In [61]:
squential_chain.run(letters)



> Entering new SimpleSequentialChain chain...
[Generated with ChatGPT]

Confidential Document - For Internal Use Only

Date: July 1, 2023
 这是一篇关于一份机密文件的总结，它的创作日期是2023年7月1日，仅供内部使用。

> Finished chain.


' 这是一篇关于一份机密文件的总结，它的创作日期是2023年7月1日，仅供内部使用。'